# Исследование надежности заемщиков.

### Шаг 1. Обзор данных

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.
Результаты исследования будут учтены при построении модели кредитного скоринга — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

**Цель исследования** — проверить гипотезы:

1. Есть ли зависимость между количеством детей и возвратом кредита в срок?
2. Есть ли зависимость между семейным положением и возвратом кредита в срок?
3. Есть ли зависимость между уровнем дохода и возвратом кредита в срок?
4. Как разные цели кредита влияют на его возврат в срок?

**Ход исследования**

Данные о поведении пользователей мы получим из файла `data.csv`. О качестве данных ничего не известно, поэтому перед проверкой гипотез нам понадобится обзор данных. 

Мы проверим данные на ошибки и оценим их влияние на исследование. Затем, на этапе предобработки мы будем искать возхможность исправить ошибки данных, мешающие нашему исследованию.
 
Таким образом, исследование пройдёт в три этапа:
 1. Обзор данных.
 2. Предобработка данных.
 3. Проверка гипотез.


In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('/datasets/data.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Итак, в таблице двенадцать столбцов. Три типа данных в столбцах — `object`, int64, float64

Согласно документации к данным:
* `children` — количество детей в семье;
* `days_employed` — общий трудовой стаж в днях;  
* `dob_years` — возраст клиента в годах;
* `education` — уровень образования клиента;
* `education_id` — идентификатор уровня образования;
* `family_status` — всемейное положение;
* `family_status_id` — идентификатор семейного положения;
* `gender` — пол клиента;
* `income_type` — тип занятости;
* `debt` — имел ли задолженность по возврату кредитов;
* `total_income` — ежемесячный доход;
* `purpose` — цель получения кредита

Количество значений в некоторых столбцах различается. Значит, в данных есть пропущенные значения.


**Выводы**

В каждой строке таблицы — данные о заёмщике. Часть колонок описывает финансовое состояние клиентов банка. Остальные данные рассказывают о самом клиенте: возраст, семейное положение, количество детей.

Предварительно мы можем утверждать, что данных достаточно для проверки гипотез. Но встречаются пропуски в данных, поэтому необходимо их устранить, а также проверить данные на аномалии и исправления. 

### Шаг 2. Предобработка данных

### Шаг 2.1 Заполнение пропусков

Проверим количество пропусков в столбцах

In [3]:
df.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

Очень интересно - пропуски обнаружились в двух колонках: days_employed и total_income, причем количество пропусков одинаковое: 2174
Возможно, проблема технического характера - при выгрузке данных могла произойти ошибка. 

Необходимо заполнить пропуски, чтобы мы смогли корректно работать с данными. 

Для колонки **total_income** заменим пропуски и будем использовать для замены медианное значение по всей этой колонке, т.к. доход — количественное значение. Пропуски в таких колонках обычно заполняют характерными значениями: средним арифметическим или медианой. В случае с ежемесячным доходом лучше использовать медиану, ведь в данных могут быть выдающиеся значения, которые сместят среднее. Заполнять пропуски нулём или максимальным значением не будем — такие значения нельзя назвать характерными.

Для колонки **days_employed** пропускам присвоим также медианное значение, т.к. значение 0 или среднее арифметическое может существенно исказить данные по наличию стажа по столбцу в целом.

Найдём медианные значения в столбцах **total_income** и **days_employed**, чтобы потом использовать эти данные для заполнения пропущенных значений. 

In [4]:
total_income_median = df['total_income'].median()
days_employed_median = df['days_employed'].median()

Заполним пропуски в столбцах **days_employed** и **total_income** на найденные выше значения

In [5]:
df['total_income'].fillna(total_income_median, inplace=True)
df['days_employed'].fillna(days_employed_median, inplace=True)

Выведем первые 30 строк таблицы, чтобы проверить наши замены:

In [6]:
df[:30]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


Замены корректны, также проверим всю таблицу на наличие пропусков, чтобы удостовериться, что все пропущенные значения были исправлены. 
Обнаружились данные в верхнем регистре в столбце education и family_status, позже приведем все значения в нём к нижнему. 

In [7]:
df.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

**Выводы**

Мы проверили, посчитали и заменили пропущенные значения в таблице. Теперь необходимо разобраться с типами данных и аномалиями в таблице и можно приступать к анализу. 

### Шаг 2.2 Проверка данных на аномалии и исправления.

Из предыдущего пункта в таблице замечены некорректные отображения данных в столбцах, например, отрицательны значения в колонке days_employed. Чтобы понять масштабы, воспользуемся методом из библиотеки pandas describe(). Этот метод используются для просмотра основных статистических данных: максимума, минимума, среднего значения и процентиля. 

In [8]:
df.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,21525.000000,21525.000000,21525.000000,21525.000000,21525.000000,2.152500e+04
mean,0.538908,56557.335698,43.293380,0.817236,0.972544,0.080883,1.651595e+05
std,1.381587,134922.319298,12.574584,0.548138,1.420324,0.272661,9.786607e+04
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,-2518.168900,33.000000,1.000000,0.000000,0.000000,1.077982e+05
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,1.000000,-385.106616,53.000000,1.000000,1.000000,0.000000,1.955436e+05
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


Исходя из описания нашей таблицы, можно сделать следующие наблюдения:
* Минимальное значение по столбцу 'children' **-1**. Не совсем ясно, почему так, т.к. минимальное значение в этой колонке подразумевает 0. Поэтому заменим все отрицательные числа в этом столбце на минимально допустимое
* Максимальное значение по столбцу 'children' **20**. Достаточно большое количество детей, необходимо взять во внимание это число и проверить, как часто оно встречается, нет ли здесь ошибки 😳
* Отрицательные значения по количеству отработанных дней 'days_employed' - мы не сможем использовать отрицательные числа для верных подсчетов, т.к. рабочий стаж не может быть отрицательным. Поэтому необходимо будет использовать модули значений в строках этого столбца. Также мы изменим тип данных с float64 на int64, т.к. подсчет общего трудового стажа заёмщиков ведётся в днях, и подсчет мы будем вести по целым дням (без дробной части)
* Среднее значение по количеству отработанных дней в 'days_employed' **63046**. Это достаточно большая цифра (около 173 лет), необходимо проверить корректность заполнения строк столбца и сообщить разработчикам, возможно, в подобных значениях стаж записан в часах. 
* Минимальное значение по столбцу 'dob_years' равно 0. Такие данные не совсем корректны, т.к. в возрасте 0 лет человек ещё не в состоянии оформить кредит 😄
* Для значений 'total_income' также изменим тип данных с float64 на int64 для наглядности и удобства расчетов. Целой части без десятичного разделителя будет достаточно для анализа

### Шаг 2.3. Изменение типов данных.

Изменим типы данных столбцов **total_income** и **days_employed** c вещественных чисел на целые и -1 на 0 в столбце **children**.
Также пройдёмся по столбцу education и 'family_status' и приведём все значения в нём к одному виду в нижнем регистре. 

In [9]:
df['days_employed'] = df['days_employed'].abs()
df['days_employed'] = df['days_employed'].astype(int)
df['total_income'] = df['total_income'].astype(int)
# использовала эту функцию, а не abs(), т.к. нужно изменить минимальное значение на ноль (отсуствие детей), а модуль дал бы неверный вариант (наличие детей)
df.loc[df['children'] == -1, 'children'] = 0 
df['education'] = df['education'].str.lower()
df['family_status'] = df['family_status'].str.lower()

Проверим результаты изменений:

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   children          21525 non-null  int64 
 1   days_employed     21525 non-null  int64 
 2   dob_years         21525 non-null  int64 
 3   education         21525 non-null  object
 4   education_id      21525 non-null  int64 
 5   family_status     21525 non-null  object
 6   family_status_id  21525 non-null  int64 
 7   gender            21525 non-null  object
 8   income_type       21525 non-null  object
 9   debt              21525 non-null  int64 
 10  total_income      21525 non-null  int64 
 11  purpose           21525 non-null  object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


In [11]:
df.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,21525.000000,21525.000000,21525.000000,21525.000000,21525.000000,2.152500e+04
mean,0.541092,60277.471731,43.293380,0.817236,0.972544,0.080883,1.651589e+05
std,1.379943,133301.576675,12.574584,0.548138,1.420324,0.272661,9.786608e+04
min,0.000000,24.000000,0.000000,0.000000,0.000000,0.000000,2.066700e+04
25%,0.000000,1025.000000,33.000000,1.000000,0.000000,0.000000,1.077980e+05
50%,0.000000,1808.000000,42.000000,1.000000,0.000000,0.000000,1.450170e+05
75%,1.000000,4779.000000,53.000000,1.000000,1.000000,0.000000,1.955430e+05
max,20.000000,401755.000000,75.000000,4.000000,4.000000,1.000000,2.265604e+06


In [12]:
df[:20]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу
5,0,926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья
6,0,2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем
7,0,152,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование
8,2,6929,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы
9,0,2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи


Все наши изменения применены успешно. Отрицательных значений в таблице также больше нет. 

### Шаг 2.4. Удаление дубликатов.

Используем методы duplcated(), чтобы найти дубликаты и sum() для их подсчета:

In [13]:
df.duplicated().sum()

71

Удалим дубликаты и проверим ещё раз - дубликаты удалены.

In [14]:
df = df.drop_duplicates()
df.duplicated().sum()

0

В колонке 'gender' есть одно значение, не подходящее в наши условия анализа:

In [16]:
df['gender'].value_counts().to_frame()

,gender
F,14174
M,7279
XNA,1


Оставим только те значения, которые мы можем использовать в данной ситуации:

In [17]:
df = df[df['gender'] != 'XNA']

Проверим столбец с возрастом. Как упоминалось выше, минимальное значение в этом столбце равно нулю, чего не должно быть, т.к. в возрасте 0 лет человек ещё не имеет возможности оформить кредит.

In [18]:
df['dob_years'].value_counts().to_frame()

,dob_years
35,616
40,607
41,605
34,601
38,597
42,596
33,581
39,572
31,559
36,554


Оказалось, что в колонке с возрастом 101 значение, равное нулю. Это достаточно много, необходимо изменить нули на подходящие по типу данные. Будем использовать информацию из колонки income_type — типу занятости. Эта информация поможет более корректно подобрать возраст. Для начала найдём медиану возрастов - она в нашем случае будет полезнее для оценки возрастных распределений, чем среднее значение.

In [19]:
income_type_median = df.groupby('income_type')['dob_years'].median()
income_type_median.astype('int').to_frame()

,dob_years
income_type,
безработный,38
в декрете,39
госслужащий,40
компаньон,39
пенсионер,60
предприниматель,42
сотрудник,39
студент,22


In [20]:
df.loc[(df['income_type'] == 'безработный ') & (df['dob_years'] == 0), 'dob_years'] = 38
df.loc[(df['income_type'] == 'в декрете') & (df['dob_years'] == 0), 'dob_years'] = 39
df.loc[(df['income_type'] == 'госслужащий') & (df['dob_years'] == 0), 'dob_years'] = 40
df.loc[(df['income_type'] == 'компаньон') & (df['dob_years'] == 0), 'dob_years'] = 39
df.loc[(df['income_type'] == 'пенсионер') & (df['dob_years'] == 0), 'dob_years'] = 60
df.loc[(df['income_type'] == 'предприниматель') & (df['dob_years'] == 0), 'dob_years'] = 42
df.loc[(df['income_type'] == 'сотрудник') & (df['dob_years'] == 0), 'dob_years'] = 39
df.loc[(df['income_type'] == 'студент') & (df['dob_years'] == 0), 'dob_years'] = 22

In [21]:
df['dob_years'].astype('int').value_counts().to_frame()

,dob_years
39,647
35,616
40,613
41,605
34,601
38,597
42,596
33,581
31,559
36,554


Все нулевые значения в столбце исправлены в соотвествии с типом занятости. Теперь проверим значения в строках столбца purpose — цель получения кредита. В нём очень много неявных дубликатов, необходимо сгруппировать их в несколько общих категорий. 

In [22]:
df['purpose'].value_counts().to_frame()

,purpose
свадьба,791
на проведение свадьбы,768
сыграть свадьбу,765
операции с недвижимостью,675
покупка коммерческой недвижимости,661
операции с жильем,652
покупка жилья для сдачи,651
операции с коммерческой недвижимостью,650
жилье,646
покупка жилья,646


Напишем функцию, которая поможет нам сгруппировать все значения целей получения кредита в 4 категории: операции с автомобилем, операции с недвижимостью, проведение свадьбы, получение образования. Такая сортировка значительно облегчит нам анализ данных в столбце. 

In [23]:
def purpose_category(row):    
    if 'автомобиль' in row:
        return 'операции с автомобилем'
    if 'свадьба' in row:
        return 'проведение свадьбы'
    if 'образование' in row:
        return 'получение образования'
    return 'операции с недвижимостью'

df['purpose_category'] = df['purpose'].apply(purpose_category)

Применили функцию, создали новый столбец и перенесли туда данные из столбца 'purpose', но уже с учетом новых категорий. Их получилось 4 - как мы и планировали:

In [24]:
df['purpose_category'].unique()

array(['операции с недвижимостью', 'получение образования',
       'операции с автомобилем', 'проведение свадьбы'], dtype=object)

### Шаг 2.5. Формирование дополнительных датафреймов словарей, декомпозиция исходного датафрейма.

Создадим функцию и столбец total_income_category с категориями:
* 0–30000 — 'E';
* 30001–50000 — 'D';
* 50001–200000 — 'C';
* 200001–1000000 — 'B';
* 1000001 и выше — 'A'.

Данные возьмём из столбца total_income.

In [25]:
def total_income_category(income):
    if 0<=income<30000:
        return 'E'
    if 30001<=income<50000:
        return 'D'
    if 50001<=income<200000:
        return 'C'
    if 200001<=income<1000000:
        return 'B'
    return 'A'

Проверим, работает ли наша функция:

In [26]:
df['total_income_category'] = df['total_income'].apply(total_income_category)
df['total_income_category'].unique()

array(['B', 'C', 'D', 'E', 'A'], dtype=object)

Всё в порядке, суммы доходов были распределены на 5 категорий. 

Теперь создадим два новых датафрейма со столбцами: 
* education_id и education — в первом education_info;
* family_status_id и family_status — во втором family_status_info.

Новые датафреймы — это те самые «словари», к которым мы сможете обращаться по идентификатору. Также удалим дубликаты в наших «словарях».

In [27]:
education_info = df[['education_id', 'education']]
education_info = education_info.drop_duplicates().sort_values(by = 'education_id').reset_index(drop = True)
family_status_info = df[['family_status_id', 'family_status']]
family_status_info = family_status_info.drop_duplicates().sort_values(by = 'family_status_id').reset_index(drop = True)

Теперь изменим вид основной таблицы, удалим столбы education и family_status, оставив только столбы с идентификаторами образования и семейного положения соответственно. Для расшифровки идентификаторов мы создали датафреймы - словари выше. 

In [28]:
df = df[['children', 'days_employed', 'dob_years', 'education_id',
       'family_status_id', 'gender', 'income_type', 'debt',
       'purpose_category', 'total_income', 'total_income_category']]

Теперь можно посмотреть новый вид нашей таблицы с данными, стало легче читать информацию о заёмщиках. 

In [29]:
df[:20]

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,purpose_category,total_income,total_income_category
0,1,8437,42,0,0,F,сотрудник,0,операции с недвижимостью,253875,B
1,1,4024,36,1,0,F,сотрудник,0,операции с недвижимостью,112080,C
2,0,5623,33,1,0,M,сотрудник,0,операции с недвижимостью,145885,C
3,3,4124,32,1,0,M,сотрудник,0,получение образования,267628,B
4,0,340266,53,1,1,F,пенсионер,0,операции с недвижимостью,158616,C
5,0,926,27,0,1,M,компаньон,0,операции с недвижимостью,255763,B
6,0,2879,43,0,0,F,компаньон,0,операции с недвижимостью,240525,B
7,0,152,50,1,0,M,сотрудник,0,получение образования,135823,C
8,2,6929,35,0,1,F,сотрудник,0,операции с недвижимостью,95856,C
9,0,2188,41,1,0,M,сотрудник,0,операции с недвижимостью,144425,C


### Проверка гипотез

#### Проверим нашу первую гипотезу: есть ли зависимость между количеством детей и возвратом кредита в срок.

Для проверки этой гипотезы мы будем работать со строками данных из столбцов с количеством детей и наличием/отсутствием просроченной задолженности. Сделаем сводную таблицу, посчитаем общее количество должников и добросовестных заёмщиков с группировкой по количеству детей:

In [30]:
df_children = df.pivot_table(index='children',columns='debt', values='total_income', aggfunc='count').fillna(0).astype('int')
df_children.columns = ['no_debt', 'debt']
df_children['%'] = df_children['debt'] / (df_children['debt'] + df_children['no_debt'])
df_children.sort_values(by='%', ascending=False).style.format({'%':'{:.1%}'})

,no_debt,debt,%
children,,,
20,68,8,10.5%
4,37,4,9.8%
2,1858,194,9.5%
1,4364,444,9.2%
3,303,27,8.2%
0,13073,1064,7.5%
5,9,0,0.0%


Исходя из данных в таблице выше, можно сделать вывод, что прослеживается корреляция между наличием просроченной задолженности и количеством(наличием/отсутствием) детей. Заёмщики без детей чаще вносят платежи по кредиту в срок, но основываясь лишь на этих данных, мы не можем сделать вывод, что непосредственно наличие/отсутствие детей и их количество у заёмщиков являются потенциальным показателем неплатежеспособности. Возможно, стоит также проанализировать эти данные, включив информацию о семейном положении и уровне дохода. 

#### Проверим вторую гипотезу: есть ли зависимость между семейным положением и возвратом кредита в срок.

Для проверки второй гипотезы создадим сводную таблицу: будем использовать данные из основного датасета и расшифровку id по семейному положению из нашего датафрейма-словаря. Для наглядности рассортируем значения колонки 'debt' по наличию/отсуствию задолженности по кредиту относительно семейного положения. Для подсчета процентного соотношения просроченных задолженностей к общему количеству кредитов создадим столбец 'ratio', в котором пропишем формулу расчета:

In [31]:
df_family = df.pivot_table(index='family_status_id', columns='debt', values='total_income', aggfunc='count').fillna(0)
df_family.columns = ['no_debt', 'debt']
df_family['ratio'] = df_family['debt'] / (df_family['debt'] + df_family['no_debt']) * 100
df_family['ratio'] = df_family['ratio'].round(1).astype(str) + '%'
df_family = df_family.merge(family_status_info, on = 'family_status_id', how = 'left')
df_family.sort_values(by='ratio', ascending = False)

,family_status_id,no_debt,debt,ratio,family_status
4,4,2536,274,9.8%,не женат / не замужем
1,1,3762,388,9.3%,гражданский брак
0,0,11408,931,7.5%,женат / замужем
3,3,1110,85,7.1%,в разводе
2,2,896,63,6.6%,вдовец / вдова


Исходя из данных в нашей таблице можно сказать, что у людей, не оформивших отношения официально, либо живущих в гражданском браке, выше процент просрочек по кредиту. Это может быть связано с тем, что в некоторых случаях банк требует согласие супруга/супруги (бывших или нынешних) при рассмотрении кредитных заявок на определенные цели кредита (зависит от внутренних регламентов). Мы также рассмотрим чуть ниже, как цели кредита влияют на возвраты кредитов в срок. Также существует вероятность дополнительных психологических критериев ответственности людей, находящихся в браке.

#### Проверим третью гипотезу: есть ли зависимость между уровнем дохода и возвратом кредита в срок.

Ранее в нашем исследовании мы разбили уровни дохода на категории от **A** до **E**, где **Е** - самый низкий уровень дохода. Сделаем сводную таблицу, в которой посмотрим количество задолженностей по каждой категории платёжеспособности и выясним их долю относительно общего числа заёмщиков в каждой категории:

In [32]:
df_income = df.pivot_table(index='total_income_category', columns='debt',values='total_income',aggfunc='count').fillna(0)
df_income.columns = ['no_debt', 'debt']
df_income['ratio'] = df_income['debt'] / (df_income['debt'] + df_income['no_debt']) * 100
df_income['ratio'] = df_income['ratio'].round(1).astype(str) + '%'
df_income.sort_values(by='ratio', ascending = False)

,no_debt,debt,ratio
total_income_category,,,
E,20,2,9.1%
C,14656,1360,8.5%
A,23,2,8.0%
B,4684,356,7.1%
D,329,21,6.0%


Напомним себе, что означают категории дохода в нашей сводной таблице:
* 0–30000 — **'E'**: низкий уровень дохода;
* 30001–50000 — **'D'**: обычный уровень дохода;
* 50001–200000 — **'C'**: средний уровень дохода;
* 200001–1000000 — **'B'**: уровень дохода выше среднего;
* 1000001 и выше — **'A'**: высокий уровень дохода. 

Исходя из данных выше, можно сказать, что чаще всего просрочки по кредитам допускают заёмщики с доходом ниже среднего, т.к. такие заёмщики чаще подвержены нехватке средств, особенно в случае возникновения у них непредвиденных ситуаций. Примерно на одном уровне по просрочкам заёмщики с высоким и среднем уровнем дохода. Заёмщики с обычным уровнем дохода и с доходом выше среднего чаще платят по кредиту вовремя.
Нужно отметить, что данная выборка не совсем репрезентативна, т.к. количество заёмщиков с высоким и низким уровнем дохода очень мало по сравнению с остальными категориями по этой сводной таблице, поэтому нет возможности оценить данные объективно. 

#### Проверим четвёртую гипотезу: как разные цели кредита влияют на его возврат в срок.

Ранее мы поделили цели кредита на 4 основных категории: **операции с автомобилем, операции с недвижимостью, получение образования и проведение свадьбы**. Сделаем сводную таблицу, в которой будет видно количество заёмщиков с просроченной задолженностью и посчитаем, какую долю составляют недобросовестные заёмщики в каждой категории целей:

In [33]:
df_purpose = df.pivot_table(index='purpose_category', columns='debt',values='total_income',aggfunc='count').fillna(0)
df_purpose.columns = ['no_debt', 'debt']
df_purpose['ratio'] = df_purpose['debt'] / (df_purpose['debt'] + df_purpose['no_debt']) * 100
df_purpose['ratio'] = df_purpose['ratio'].round(1).astype(str) + '%'
df_purpose.sort_values(by='ratio', ascending = False)

,no_debt,debt,ratio
purpose_category,,,
операции с автомобилем,882,90,9.3%
получение образования,2463,236,8.7%
проведение свадьбы,727,64,8.1%
операции с недвижимостью,15640,1351,8.0%


Опираясь на данные этой таблицы, можно сказать, что меньше всего просрочек по кредиту допускают заёмщики, использующие кредитные средства на операции с недвижимостью. Можно это связать с более высоким уровнем ответственности у заёмщиков и более требовательным регламентом по проверке предоставляемых банку документов и доказательств платёжеспособности, т.к. недвижимость является важным критерием уровня жизни. 

### Ответы на вопросы.

##### Вопрос 1: Есть ли зависимость между количеством детей и возвратом кредита в срок?

##### Вывод 1:
Прослеживается корреляция между наличием просроченной задолженности и количеством(наличием/отсутствием) детей. Заёмщики без детей чаще вносят платежи по кредиту в срок, но основываясь лишь на этих данных, мы не можем сделать вывод, что непосредственно наличие/отсутствие детей и их количество у заёмщиков являются потенциальным показателем неплатежеспособности. Возможно, стоит также проанализировать эти данные, включив информацию о семейном положении и уровне дохода. 

| Количество детей (0 - отсутствие) | нет просроченной задолженности | просроченная задолженность | % просрочек от общего количества по категории |
|---:|---:|---:|---:|
| 20 | 68 | 8 | 10.5% |
| 4 | 37 | 4 | 9.8% |
| 2 | 1858 | 194 | 9.5% |
| 1 | 4364 | 444 | 9.2% |
| 3 | 303 | 27 | 8.2% |
| 0 | 13073 | 1064 | 7.5% |
| 5 | 9 | 0 | 0.0% |

##### Вопрос 2: Есть ли зависимость между семейным положением и возвратом кредита в срок?

##### Вывод 2:
У заёмщиков, не оформивших отношения официально, либо живущих в гражданском браке, выше процент просрочек по кредиту. Это может быть связано с тем, что в некоторых случаях банк требует согласие супруга/супруги (бывших или нынешних) при рассмотрении кредитных заявок на определенные цели кредита (зависит от внутренних регламентов). Также стоит отметить, что обеспечение возвратности кредита сложнее у заёмщиков, не состоящих в официальных отношениях, в силу юридических и законодательных норм, например, как в официальных супружеских взаимоотношениях.

| семейное положение | нет просроченной задолженности | просроченная задолженность | % просрочек от общего количества по категории |
|---|---:|---:|---:|
| не женат / не замужем | 2536 | 274 | 9.8% |
| гражданский брак | 3762 | 388 | 9.3% |
| женат / замужем | 11408 | 931 | 7.5% |
| в разводе | 1110 | 85 | 7.1% |
| вдовец / вдова | 896 | 63 | 6.6% |

##### Вопрос 3: Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

##### Вывод 3:
Чаще всего просрочки по кредитам допускают заёмщики с доходом ниже среднего, т.к. такие заёмщики чаще подвержены нехватке средств, особенно в случае возникновения у них непредвиденных ситуаций. Примерно на одном уровне по просрочкам заёмщики с высоким и среднем уровнем дохода. Заёмщики с обычным уровнем дохода и с доходом выше среднего чаще платят по кредиту вовремя. Нужно отметить, что данная выборка не совсем репрезентативна, т.к. количество заёмщиков с высоким и низким уровнем дохода очень мало по сравнению с остальными категориями по этой сводной таблице, поэтому нет возможности оценить данные объективно.

| категории дохода | нет просроченной задолженности | просроченная задолженность | % просрочек от общего количества по категории |
|---:|---:|---:|---:|
| низкий уровень дохода | 20 | 2 | 9.1% |
| средний уровень дохода | 14656 | 1360 | 8.5% |
| высокий уровень дохода | 23 | 2 | 8.0% |
| уровень дохода выше среднего | 4684 | 356 | 7.1% |
| обычный уровень дохода | 329 | 21 | 6.0% |

##### Вопрос 4: Как разные цели кредита влияют на его возврат в срок?

##### Вывод 4:
Реже всех просрочки по кредиту допускают заёмщики, использующие кредитные средства на операции с недвижимостью. Можно это связать с более высоким уровнем ответственности у заёмщиков и более требовательным регламентом по проверке предоставляемых банку документов и доказательств платёжеспособности, т.к. недвижимость является важным критерием уровня жизни. Ниже можно ознакомиться со сводной таблицей, отражающей резуьтаты исследования гипотезы:

	
| цель кредита | нет просроченной задолженности | просроченная задолженность | % просрочек от общего количества по категории |
|---|---|---|---|
| операции с автомобилем | 882 | 90 | 9.3% |
| получение образования | 2463 | 236 | 8.7% |
| проведение свадьбы | 727 | 64 | 8.1% |
| операции с недвижимостью | 15640 | 1351 | 8.0% |
 

### Общий вывод:

В нашем исследовании мы проверили, посчитали и заменили пропущенные значения в исходном датасете:

* при предобработке данных в колонках с информацией о трудовом стаже и доходе были найдены пропущенные значения и заполнены медианным значением по соответсвующему столбцу;
* исправлены отрицательные значения дней трудового стажа в столбце days_employed
* заменены вещественный тип данных в столбцах total_income и 'days_employed' на целочисленный
* удалены строки-дубликаты, обработаны неявные дубликаты например, в столбце education были одни и те же значения, но записанные по-разному: с использованием заглавных и строчных букв - такие значения приведены к одному регистру.
* созданы два новых датафрейма с информацией об образовании и семейном положении - эти датафреймы были использованы как "словари" для проверки наших гипотез
* собраны в категории данные по уровню дохода и целям кредита: эти данные использованы в исследовании

Исходя из данных, предоставленных в исследовании, можно сделать общий вывод о критериях потенциального добросовестного заёмщика:

* состоит в официальном браке;
* имеет детей;
* имеет средний уровень дохода;
* использует кредитные средства на покупку недвижимости. 

Вывод выше не означает, что все кто попадает под описанные параметры обязательно будут допускать просрочки по кредиту. К заемщикам, не попадающим полностью или частично под категории выше, требуется уделить больше внимания, анализируя заемщика комплексно. 